### Tools
Models can request to call tools that perform tasks such as fetching data from a database, searching the web, or running code. Tools are pairings of:
1. A schema, including the name of the tool, a description, and/or argument definitions (often a JSON schema)
2. A function or coroutine to execute.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain.chat_models import init_chat_model

/home/aniruddha/Projects/RAG/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
model = init_chat_model("openai:gpt-5-mini-2025-08-07")
response = model.invoke("What are fighter plane generations?")
response

AIMessage(content='"Fighter generations" is a shorthand used to group combat aircraft by their dominant technologies, design features, and the tactical roles they were built to fill. The classification isn’t official or perfectly consistent between countries or authors, but it’s useful for seeing how fighter capabilities evolved from the early-jet era to the present and near-future. Below is a compact summary of commonly referenced generations, their key characteristics, timeframe, and representative aircraft.\n\n1st generation (late 1940s–1950s)\n- Main features: Early turbojet engines, straight or mildly swept wings, basic guns and early unguided rockets, primitive radar (or none), manual flight controls.\n- Role: Replace piston fighters with jet propulsion; air-to-air combat still visual-range.\n- Examples: Messerschmitt Me 262 (first operational jet), Gloster Meteor, Lockheed P-80/F-80 Shooting Star.\n\n2nd generation (1950s–early 1960s)\n- Main features: Supersonic flight capabili

In [7]:
from langchain.tools import tool

@tool
def get_weather(location: str)->str:
    """Get the weather at a location"""
    return f"It's sunny in {location}"

model_with_tools = model.bind_tools(
    [get_weather]
)

In [10]:
response_tool = model_with_tools.invoke(
    "What's the weather like in Mumbai?"
)

print(response_tool)

for tool_call in response_tool.tool_calls:
     # View tool calls made by the model
    print(f"Tool: {tool_call['name']}")
    print(f"Args: {tool_call['args']}")

content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 132, 'total_tokens': 155, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CtVd8lswQr7fr2X2J8iqyxZU3SY2C', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019b7e02-a794-7923-977f-658f72b6813e-0' tool_calls=[{'name': 'get_weather', 'args': {'location': 'Mumbai'}, 'id': 'call_eJH2fHSiKjV809yHi1Fah1Do', 'type': 'tool_call'}] usage_metadata={'input_tokens': 132, 'output_tokens': 23, 'total_tokens': 155, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
Tool: get_weather
Args: {'location': 'Mumbai'}


### Tool Execution Loops

In [11]:
# Step 1: Model generates tool calls
messages = [{"role": "user", "content": "What's the weather in Mumbai?"}]
ai_message = model_with_tools.invoke(messages)
messages.append(ai_message)

# Step 2: Execute tools and collect results
for tool_call in ai_message.tool_calls:
    # Execute the tool with the generated arguments
    tool_result = get_weather.invoke(tool_call)
    messages.append(tool_result)

# Step 3: Pass results back to model for final response
final_response = model_with_tools.invoke(messages)
print(final_response.text)

It's sunny in Mumbai. Would you like the current temperature, humidity, or a short forecast for the next few hours?


In [12]:
messages

[{'role': 'user', 'content': "What's the weather in Mumbai?"},
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 131, 'total_tokens': 154, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CtVkEQfSzXa7DSwoN3HyBZMUW7Mw1', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b7e09-5e2d-7842-b94a-9e79d8859eac-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'Mumbai'}, 'id': 'call_rCNzst4bC8mhC78fdDjgSuMc', 'type': 'tool_call'}], usage_metadata={'input_tokens': 131, 'output_tokens': 23, 'total_tokens': 154, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio'